In [1]:
# !pip install transformers
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from datetime import datetime
import numpy as np


gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

time = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-uncased-v1")

In [2]:
texts, labels, files = [], [], ["FinnSentiment2020neg", "FinnSentiment2020neut", "FinnSentiment2020pos"]
for i, file in enumerate(files):
    with open(f"E:/Emodim/data/tr/{file}.txt", "r") as f:
        lines = f.read().splitlines()
        texts.extend(lines), labels.extend([i]*len(lines))

In [3]:
token_lens = []
for txt in texts:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
max_length=np.max(token_lens)
print(max_length)
MAX_LEN = max_length + 2

196


In [4]:
from sklearn.model_selection import train_test_split


train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=.1)


def tokenize(data, max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(data[i], add_special_tokens=True, max_length=max_len, padding='max_length', return_attention_mask=True)
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids), np.array(attention_masks)

train_input_ids, train_attention_masks = tokenize(train_texts, MAX_LEN)
val_input_ids, val_attention_masks = tokenize(val_texts, MAX_LEN)

In [5]:
'''from sklearn.model_selection import train_test_split


train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=.1)
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
# train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors=tf)
# val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors=tf)

train_dataset = tokenizer(train_texts, truncation=True, padding=True, return_tensors="tf")
val_dataset = tokenizer(val_texts, truncation=True, padding=True, return_tensors="tf")
train_dataset = train_dataset.data
val_dataset = val_dataset.data
# train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
# val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings),val_labels))


train_encodings = tokenizer.batch_encode_plus(train_texts,
                                              max_length=MAX_SEQUENCE_LENGTH, # set the length of the sequences
                                              add_special_tokens=True, # add [CLS] and [SEP] tokens
                                              return_attention_mask=True,
                                              return_token_type_ids=False, # not needed for this type of ML task
                                              truncation=True,
                                              padding='longest', # add 0 pad tokens to the sequences less than max_length
                                              return_tensors='tf')
val_encodings = tokenizer.batch_encode_plus(val_texts,
                                            max_length=MAX_SEQUENCE_LENGTH, # set the length of the sequences
                                            add_special_tokens=True, # add [CLS] and [SEP] tokens
                                            return_attention_mask=True,
                                            return_token_type_ids=False, # not needed for this type of ML task
                                            truncation=True,
                                            padding='longest', # add 0 pad tokens to the sequences less than max_length
                                            return_tensors='tf')
'''

'from sklearn.model_selection import train_test_split\n\n\ntrain_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=.1)\ntrain_labels = np.array(train_labels)\nval_labels = np.array(val_labels)\n# train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors=tf)\n# val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors=tf)\n\ntrain_dataset = tokenizer(train_texts, truncation=True, padding=True, return_tensors="tf")\nval_dataset = tokenizer(val_texts, truncation=True, padding=True, return_tensors="tf")\ntrain_dataset = train_dataset.data\nval_dataset = val_dataset.data\n# train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))\n# val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings),val_labels))\n\n\ntrain_encodings = tokenizer.batch_encode_plus(train_texts,\n                                              max_length=MAX_SEQUENCE_LENGTH, # set the leng

In [6]:
from transformers import TFBertForSequenceClassification


bert_model = TFBertForSequenceClassification.from_pretrained("TurkuNLP/bert-base-finnish-uncased-v1")
# bert_model = TFBertForSequenceClassification.from_pretrained("TurkuNLP/bert-base-finnish-uncased-v1", output_hidden_states=True, num_labels=3)
#input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_ids')
#attention_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='attention_mask')
#output = bert_model([input_ids, attention_mask])[0]
#output = tf.keras.layers.Dense(3, activation='softmax')(output)
#model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)
# print(model.summary())

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at TurkuNLP/bert-base-finnish-uncased-v1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
"""
def build_classifier_model():
    input_ids = tf.keras.Input(shape=(128, ),dtype='int32')
    attention_mask = tf.keras.Input(shape=(128, ), dtype='int32')

    transformer = model([input_ids, attention_mask])    
    hidden_states = transformer[1] # get output_hidden_states
    
    hidden_states_size = 4 # count of the last states 
    hiddes_states_ind = list(range(-hidden_states_size, 0, 1))

    selected_hiddes_states = tf.keras.layers.concatenate(tuple([hidden_states[i] for i in hiddes_states_ind]))

    X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(net)
    X = tf.keras.layers.Concatenate(axis=-1)([X, input_layer])
    X = tf.keras.layers.MaxPooling1D(20)(X)
    X = tf.keras.layers.SpatialDropout1D(0.4)(X)
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(128, activation="relu")(X)
    X = tf.keras.layers.Dropout(0.25)(X)
    X = tf.keras.layers.Dense(3, activation='softmax')(X)

    model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)
        for layer in model.layers[:3]:
        layer.trainable = False
    return model
"""

import tensorflow_addons as tfa


def create_model(bert_model, max_len=MAX_LEN):
    
    opt = tfa.optimizers.AdamW(learning_rate=1e-5, weight_decay=1e-7)
    # opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    input_ids = tf.keras.Input(shape=(max_len,), dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,), dtype='int32')
    embeddings = bert_model.bert([input_ids, attention_masks])[1]
    output = tf.keras.layers.Dense(3, activation="softmax")(embeddings)
    model = tf.keras.models.Model(inputs = [input_ids, attention_masks], outputs = output)
    model.compile(opt, loss=loss, metrics=accuracy)
    
    
    return model

In [35]:
model = create_model(bert_model, MAX_LEN)
# model.summary()

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
history_bert = model.fit([train_input_ids, train_attention_masks], np.array(train_labels), 
                         validation_data=([val_input_ids, val_attention_masks], np.array(val_labels)), 
                         epochs=4, batch_size=64, callbacks=[early_stop])

Epoch 1/4


ResourceExhaustedError: Graph execution error:

Detected at node 'model_7/bert/encoder/layer_._4/attention/self/Softmax' defined at (most recent call last):
    File "E:\Miniconda\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "E:\Miniconda\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "E:\Miniconda\envs\tf\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "E:\Miniconda\envs\tf\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "E:\Miniconda\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "E:\Miniconda\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "E:\Miniconda\envs\tf\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_once

In [ ]:
"""import tensorflow_addons as tfa


model.compile(optimizer = tfa.optimizers.AdamW(learning_rate=5e-4, weight_decay=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
              metrics = tf.keras.metrics.SparseCategoricalAccuracy())
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
# model.fit(train_dataset.shuffle(1000).batch(16), callbacks=[early_stop], epochs=5, batch_size=16)
history = model.fit(train_dataset, validation_data=val_dataset, callbacks=[early_stop], epochs=5)

history = model.fit(x=train_encodings.values(),
                    y=np.array(train_labels), 
                    validation_data=(val_encodings.values(), np.array(val_labels)),  
                    callbacks=[early_stop], 
                    epochs=5)"""

In [ ]:
modelpath = f'D:\Work\Models\fine_tuned_finBERT_tf_{time}_2\\1'
# modelpath = f'E:\Emodim\data\others\\fine_tuned_finBERT_tf_14-06-2022_12-00-22\\1'
model.save(modelpath)

In [ ]:
import_model = tf.keras.models.load_model(f"{modelpath}")
import_model.summary()

In [ ]:
import random
import numpy as np
n = 50
inputs = []
with open(f"D:\\Work\\Data\\s24_2001_sentences_shuffled_slice.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()
    random.shuffle(lines)
    for line in lines:
        inputs.append(line.strip('\n'))

In [ ]:
preds = []
for inp in inputs[:n]:
    input_text_tokenized = tokenizer.encode(inp, truncation=True, padding=True, return_tensors="tf")
    prediction = import_model(input_text_tokenized)
    prediction_logits = prediction[0]
    prediction_probs = tf.nn.softmax(prediction_logits,axis=1).numpy()
    print(f'Sentence: {inp}, Probabilities: {prediction_probs}')

In [ ]:
preds = []
for inp in inputs[:n]:
    text = tokenizer.encode(inp,
                          truncation=True,
                          padding=True,
                          return_tensors="tf")
    print(inp, text)
    prediction = model(text)
    prediction_logits = prediction[0]
    prediction_probs = tf.nn.softmax(prediction_logits,axis=1).numpy()
    print(f'Sentence: {inp}, Probabilities: {prediction_probs}')